# Challenge 2: Exploratory Data Analysis (EDA)

In [89]:
# Data manipulation
import pandas as pd # data manipulation and dataframes
import numpy as np # arrays manipulation and mathematical operations

# Fuzzy string matching
from fuzzywuzzy import fuzz

In [90]:
# Pandas configuration
pd.set_option('display.max_columns', None)  # shows all columns
pd.set_option('display.max_colwidth', None)  # shows all cell content

## Some cleaning

In [91]:
# Reads csvs
initial_csv = pd.read_csv('../data/initial-dia.csv', index_col = 0)
scrapped_csv = pd.read_csv('../data/scraped-dia.csv', index_col = 0)

# Index
initial_csv.reset_index(drop = False, inplace = True) # resets index named by url

# Nulls
initial_csv.isnull().sum() # 3053473 description, 5480 reference_unit

# Drops
initial_csv.drop(columns=['description'], axis = 1, inplace = True) # descrition column (all values are null)
initial_csv.drop(columns=['product_id'], axis = 1, inplace = True) # product_id column (not used information)
#initial_csv.drop(columns=['url'], axis = 1, inplace = True) # url column (not used information)
scrapped_csv.drop(columns=['subcategory'], axis = 1, inplace = True) # subcategory column (not used information)

# Gets YYYY-MM-DD format
initial_csv['insert_date'] = initial_csv['insert_date'].str.split(' ', expand = True).get(0) 

# Matches reference_unit column content between both dfs
units = scrapped_csv['reference_unit'].tolist()

new_units = []

for u in units:

    if u == 'kilo':
        new_units.append('kg')

    elif u == 'unidad':
        new_units.append('ud')

    elif u == 'litro':
        new_units.append('l')

    elif u == 'metro':
        new_units.append('m')

    elif u == 'lavado':
        new_units.append('lavado')

    else:
        new_units.append(np.nan)

scrapped_csv['reference_unit'] = new_units

# Adjusts column dtypes
initial_csv['insert_date'] = pd.to_datetime(initial_csv['insert_date']) # to datetime
scrapped_csv['insert_date'] = pd.to_datetime(scrapped_csv['insert_date']) # to datetime
scrapped_csv.iloc[2698, scrapped_csv.columns.get_loc('reference_price')] = '13.82' # corrects supermarket labelling error
scrapped_csv['reference_price'] = pd.to_numeric(scrapped_csv['reference_price']) # to float

# No url info for scraped csv
scrapped_csv['url'] = '' # wip

# Same column order
scrapped_csv = scrapped_csv.reindex(columns = initial_csv.columns)

## Matching csvs by category with FuzzyWuzzy

In [92]:
# Category column from both csvs to list
new_categories = scrapped_csv['category'].unique().tolist()
old_categories = initial_csv['category'].unique().tolist()

def best_match(col, new_categories_key_words):
    '''
    Finds most likely match between categories after applying FuzzyWuzzy
    Receives 2 arguments:
        col (pd.Series): category column from initial csv
        new_categories_key_words (list): list with category column key words from scraped csv
    Returns 1 argument:
        category (str): most likely match between categories after applying FuzzyWuzzy
    '''
    max_ratio = 0

    for w in new_categories_key_words:

        ratio = fuzz.ratio(col, w)

        if ratio > max_ratio:

            max_ratio = ratio
            
            category = w

    return category

# Splits scraped categories by '_'
splitted_new = []

for n in new_categories:

    splitted_new.append(n.split('_'))

# Flats splitted list
flat = []

for sublist in splitted_new:

    for item in sublist:

        flat.append(item)

# Appends key words from new categories after discarting conjunctions
new_categories_key_words = []

for f in flat:

    if f == 'y' or f == 'e' or f == 'con':

        pass

    else:

        new_categories_key_words.append(f)

# Appends special categories as navidad and difficult terms to classify
words_to_append = ['gluten', 'lacteos', 'navidad', 'dieteticos', 'solidario', 'aceitunas', 'sal', 
            'cuidado', 'internacional', 'mermeladas', 'licores', 'sopas', 'espumosos']

for a in words_to_append:
    
    new_categories_key_words.append(a)

# Splits initial categories by '_'
splitted_old =[]

for o in old_categories:

    splitted_old.append(o.split('_'))


# Discards difficult terms to classify
old_categories_key_words = []
words = ''

words_to_discard = ['y', 'e', 'vinagre', 'con', 'del', 'al', 'de', 'dia', 'alimentacion', 'fresco', 'frescos', 
            'desayuno', 'despensa', 'bodega', 'drogueria', 'desayunos', 'dulces',
            'cocina', 'personal', 'soy', 'bano', 'corporal', 'preparacion', 'bebidas']

for sublist in splitted_old:

    for item in sublist:
        
        if item in words_to_discard:

            pass

        else:

            words += item + ' '
    
    old_categories_key_words.append(words.rstrip(' '))

    words = ''

# Picks first key word from old categories
old_first_key_word = []

for w in old_categories_key_words:

    old_first_key_word.append(w.split(' ')[0])

# Generates a DataFrame with de most representative word from initial categories and the FuzzyWuzzy math result for the scraped categories
df = pd.DataFrame()
df['old_categories_key_words'] = pd.DataFrame(old_first_key_word)
df['fuzzy_new_categories_key_words'] = df.apply(lambda x: best_match(x['old_categories_key_words'], new_categories_key_words), axis = 1)

In [93]:
# After matching the categories by the most representative word it is neccesary to go back to original 'category names'

# Appends key words from new categories after discarting conjunctions as before but in a list of lists (not in a flat list)
key_words_list_new_categories = []
aux = []

for sublist in splitted_new:

    for f in sublist:

        if f == 'y' or f == 'e' or f == 'con':

            pass

        else:
            
            aux.append(f)
            
    key_words_list_new_categories.append(aux)
    aux =[]

def list_to_dict(words, category_key):
    '''
    Establish a relation between category name (key) and the list of the most representative words for that category (values)
    Receives 2 arguments:
        words (list): list of list of the most representative words for each category
        category_key (str): category name from scrapped csv
    Returns 1 argument:
        dict_category_key_words (dict): category name: list of representative words
    '''
    dict_category_key_words = {}
    
    for i, sublist in enumerate(words):

        dict_category_key_words[category_key[i]] = sublist
    
    return dict_category_key_words

# example {azucar_chocolates_y_caramelos}: [azucar, chocolates, caramelos]
dict_category_key_words = list_to_dict(key_words_list_new_categories, new_categories)

# Relates special categories as navidad and difficult terms to classify to the category names from the scraped csv (keys)

# Special categories
dict_category_key_words['gluten'] = ['gluten'] # sorry celiacos
dict_category_key_words['solidario'] = ['solidario']
dict_category_key_words['navidad'] = ['navidad']
dict_category_key_words['internacional'] = ['internacional']

# Difficult terms to classify
dict_category_key_words['leche_huevos_y_mantequilla'].append('lacteos')
dict_category_key_words['galletas_bollos_y_cereales'].append('dieteticos')
dict_category_key_words['patatas_fritas_encurtidos_y_frutos_secos'].append('aceitunas')
dict_category_key_words['aceites_salsas_y_especias'].append('sal')
dict_category_key_words['perfumeria_higiene_salud'].append('cuidado')
dict_category_key_words['azucar_chocolates_y_caramelos'].append('mermeladas')
dict_category_key_words['cervezas_vinos_y_bebidas_con_alcohol'].append('licores')
dict_category_key_words['conservas_caldos_y_cremas'].append('sopas')
dict_category_key_words['cervezas_vinos_y_bebidas_con_alcohol'].append('espumosos')

# Transforms most representative words used by FuzzyWuzzy to category names from the scraped csv
# example leche: leche_huevos_y_mantequilla
lst = []

for i in df['fuzzy_new_categories_key_words']:

    for key, values in dict_category_key_words.items():

        if i in values:

            lst.append(key) 

# Adds to DataFrame
df['old_categories'] = old_categories
df['new_categories'] = lst

news = df['new_categories'].tolist()
olds = df['old_categories'].unique().tolist()

# Relates old categories (initial csv) to new categories (scraped csv)
# {old}: new
hada_dict = {}

for i in range(0, len(news)):

    hada_dict[olds[i]] = news[i]

# Map
initial_csv['new_categories'] = initial_csv['category'].map(hada_dict)

# Drop old category column and rename
initial_csv.drop(columns=['category'], axis = 1, inplace = True)
initial_csv.rename(columns={"new_categories": "category"}, inplace=True)

# Same column order
initial_csv = initial_csv.reindex(columns = scrapped_csv.columns)